**US House of Representatives Analysis**

*Objectives:*
1) Load Collected Data from API for Analysis ("https://house-stock-watcher-data.s3-us-west-2.amazonaws.com/data/all_transactions.json")
2) Parse and Clean
3) Better understand and brainstorm analysis ideas.

*Assumptions:*

Prior to this full Data Cleaning, we grouped company names with similar stock tickers to avoid discrepancies and facilitate visualization.

For the company Name we grouped Class A, Class B, Class C, Sponsored, Unsponsored, Bought by mistake and then sold, as well as Ordinary share purchases all under the same Company Name.

Fixed Dates to same format, and corrected blank company names based on stock ticker. 

We have also exclude transactions from 2018 given there were too few to make an insightfull impact.

More notably out of all this assumptions is the fact thaat the API provides a range of quantity invested instead of the real dollar amount that the representatives disclosed, we chose to take the upper range of this ranges to perform quantifiable transformations used on our visualization. 

In [54]:
#load dependencies
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

*Data Collection and Exploration*

In [55]:
#load dataset
data = pd.read_csv("PreCleaning_USA_Congress_Transactions_data.csv")
print(data.head(3))

             amount             asset_description  cap_gains_over_200_usd  \
0  $1,001 - $15,000        3D Systems Corporation                   False   
1  $1,001 - $15,000  3i Group plc Unsponsored ADR                   False   
2  $1,001 - $15,000                    3M Company                   False   

  disclosure_date  disclosure_year district owner  \
0       1/28/2021             2021     LA06   NaN   
1       3/16/2022             2022     FL04   NaN   
2       6/15/2020             2020     MN03   NaN   

                                            ptr_link        representative  \
0  https://disclosures-clerk.house.gov/public_dis...    Hon. Garret Graves   
1  https://disclosures-clerk.house.gov/public_dis...  Hon. John Rutherford   
2  https://disclosures-clerk.house.gov/public_dis...    Hon. Dean Phillips   

  ticker transaction_date       type  
0    DDD        1/27/2021  sale_full  
1  TGOPY        2/17/2022   purchase  
2    MMM        5/13/2020   purchase  


In [56]:
print(data.columns)
print(data.shape)

Index(['amount', 'asset_description', 'cap_gains_over_200_usd',
       'disclosure_date', 'disclosure_year', 'district', 'owner', 'ptr_link',
       'representative', 'ticker', 'transaction_date', 'type'],
      dtype='object')
(15433, 12)


*Data Transformation and Validation*

In [57]:
#Analyze uniqueness or rows 
print(data.amount.unique())

['$1,001 - $15,000' '$15,001 - $50,000' '$100,001 - $250,000'
 '$50,001 - $100,000' '$500,001 - $1,000,000' '$250,001 - $500,000'
 '$1,001 -' '$1,000,001 - $5,000,000' '$15,000 - $50,000'
 '$1,000 - $15,000' '$1,000,000 +' '$1,000,000 - $5,000,000'
 '$5,000,001 - $25,000,000' '$50,000,000 +']


There is values in the amount column starting with a 1 dollar difference
'$1,000 - $15,000' and '$1,001 - $15,000' should be the same value 
lets fix for the difference across the entire column 

In [58]:
#Fix amount column
for i in range(0,len(data)):
    index = data['amount'][i].find('01')
    if index != -1:
      data['amount'][i] = list(data['amount'][i])
      data['amount'][i][index:index+2] = '00'
      data['amount'][i] = ''.join(data['amount'][i])

In [59]:
print(data.amount.unique())
print(data.shape)

['$1,000 - $15,000' '$15,000 - $50,000' '$100,000 - $250,000'
 '$50,000 - $100,000' '$500,000 - $1,000,000' '$250,000 - $500,000'
 '$1,000 -' '$1,000,000 - $5,000,000' '$1,000,000 +'
 '$5,000,000 - $25,000,000' '$50,000,000 +']
(15433, 12)


In [60]:
#Modify representative format for easier assimilation to other data sets 
#Remove Hon. Prefix before representative name 
for i in range(0,len(data)):
    #grab last part of prefix split (First and Last Name )
    first_last_name = data['representative'][i].split(" ")[1:]
    data['representative'][i] = ' '.join(first_last_name)
data['representative'].unique()

array(['Garret Graves', 'John Rutherford', 'Dean Phillips',
       'John A. Yarmuth', 'Gilbert Cisneros', 'Cindy Axne',
       'Richard W. Allen', 'K. Michael Conaway', 'Maria Elvira Salazar',
       'Tom Malinowski', 'Earl Blumenauer', 'Donna Shalala',
       'Greg Gianforte', 'David B. McKinley', 'Joe Courtney',
       'Katherine M. Clark', 'Kevin Hern', 'John Curtis', 'Peter Meijer',
       'Rohit Khanna', 'Robert J. Wittman', 'Mikie Sherrill',
       'Josh Gottheimer', 'Carol Devine Miller', 'Michael C. Burgess',
       'Kim Dr Schrier', 'Bob Gibbs', 'Dwight Evans', 'Adam B. Schiff',
       'Zoe Lofgren', 'Marjorie Taylor Greene', 'Debbie Dingell',
       'William R. Timmons', 'Kenny Marchant', 'Kathy Manning',
       'Daniel Meuser', 'Kurt Schrader', 'Joseph D. Morelle',
       'Steve Chabot', 'Christopher L. Jacobs', 'Mark Dr Green',
       'Michael T. McCaul', 'Sara Jacobs', 'Donald Sternoff Beyer',
       'Lois Frankel', 'Francis Rooney', 'William R. Keating',
       'Susie Lee

In [61]:
#Delete rows with incomplete data 
data = data.dropna()
print(data.shape)

(9523, 12)


In [62]:
#Analysis distric, representative, and type 
data.district.unique()

array(['CA39', 'IA03', 'GA12', 'OR03', 'FL27', 'MT00', 'WV01', 'CT02',
       'MA05', 'OK01', 'CA17', 'NJ05', 'WV03', 'WA08', 'VA01', 'CA19',
       'GA14', 'MI12', 'SC04', 'TX24', 'PA09', 'NY25', 'NC06', 'TN07',
       'TX10', 'VA08', 'FL21', 'NV03', 'UT03', 'IL03', 'CA47', 'AL05',
       'NC02', 'KS01', 'NC05', 'MN03', 'KY03', 'CA52', 'WI08', 'CA53',
       'PA03', 'TX32', 'VA02', 'CO07', 'IA01', 'RI02', 'CA12', 'OK04',
       'IL10', 'NY27', 'FL15', 'IL17', 'NC07', 'CA48', 'IN05', 'TN01',
       'IL16', 'FL14', 'TX11', 'VA04', 'TX17', 'CA28', 'MD06', 'WA01',
       'MO02', 'FL04', 'AR02', 'MS03', 'TN08', 'GA08', 'TN09', 'GA01',
       'TX03', 'FL02', 'CA38', 'FL18', 'NY08', 'CA03', 'MI06', 'PA16',
       'AZ01', 'AZ03', 'NJ06', 'FL16', 'VA11', 'TX35', 'IN01', 'VT00',
       'TN03', 'PA05', 'FL23', 'OH07', 'OH05', 'NY12', 'ID02', 'FL12',
       'IN03', 'MA03', 'SC07', 'NJ09', 'NC04', 'KS04', 'CA06', 'VA07',
       'NY06', 'OH16', 'MA04', 'FL25', 'DC00', 'CO05', 'MD08', 'MA06',
      

In [63]:
data.representative.unique()

array(['Gilbert Cisneros', 'Cindy Axne', 'Richard W. Allen',
       'Earl Blumenauer', 'Donna Shalala', 'Greg Gianforte',
       'David B. McKinley', 'Joe Courtney', 'Katherine M. Clark',
       'Kevin Hern', 'Rohit Khanna', 'Josh Gottheimer',
       'Carol Devine Miller', 'Kim Dr Schrier', 'Robert J. Wittman',
       'Zoe Lofgren', 'Marjorie Taylor Greene', 'Debbie Dingell',
       'William R. Timmons', 'Kenny Marchant', 'Daniel Meuser',
       'Joseph D. Morelle', 'Kathy Manning', 'Mark Dr Green',
       'Michael T. McCaul', 'Donald Sternoff Beyer', 'Lois Frankel',
       'Susie Lee', 'John Curtis', 'Marie Newman', 'Alan S. Lowenthal',
       'Mo Brooks', 'Deborah K. Ross', 'Roger W. Marshall',
       'Virginia Foxx', 'Dean Phillips',
       'Donald Sternoff Honorable Beyer', 'John A. Yarmuth',
       'Scott H. Peters', 'Michael John Gallagher', 'Susan A. Davis',
       'Dwight Evans', 'Pete Sessions', 'Elaine Luria', 'Ed Perlmutter',
       'Ashley Hinson Arenholz', 'James R. Langev

In [64]:
data.type.unique()

array(['sale_full', 'purchase', 'sale_partial', 'exchange', 'sale'],
      dtype=object)

In [65]:
#Check for duplicate rows 
data.drop_duplicates()
print(data.shape)

(9523, 12)


*Analysis and Visualization*

In [66]:
data.to_csv(".\Cleaned_House_Stock_US.csv",index=False)

Now having cleaned this dataset I would continue my analysis on Tableu Public.

I will search for a dataset of the political parties of each representative to compare Republican and Democratic Investments. 

https://en.wikipedia.org/wiki/List_of_current_members_of_the_United_States_House_of_Representatives

However, this list only contains current representatives; for my dataset to be complete I would have to limit my dataset to last two years of office 2021-2022

Before proceeding I would clean up this data set as well prior to analysis.


**List of Current USA House Representatives**

In [67]:
reps = pd.read_csv("List_of_Representatives.csv")
reps.head()


,District,Member,Party,Prior experience,Education,Assumed office,Residence,Born[2]
0,Alabama 1,Jerry Carl,Republican,Mobile County Commission,Florida Gateway College,2021,Mobile,"June 17, 1958(age 64)"
1,Alabama 2,Barry Moore,Republican,Alabama House of Representatives,Enterprise State Community College (AS)\r\nAub...,2021,Enterprise,"September 26, 1966(age 56)"
2,Alabama 3,Mike Rogers,Republican,Calhoun County Commissioner\r\nAlabama House o...,"Jacksonville State University (BA, MPA)\r\nBir...",2003,Anniston,"July 16, 1958(age 64)"
3,Alabama 4,Robert Aderholt,Republican,Haleyville Municipal Judge,University of North Alabama\r\nBirmingham–Sout...,1997,Haleyville,"July 22, 1965(age 57)"
4,Alabama 5,Mo Brooks,Republican,Alabama House of Representatives\r\nMadison Co...,Duke University (BA)\r\nUniversity of Alabama ...,2011,Huntsville,"April 29, 1954(age 68)"


We are only interested in the representatives' political party, District, and age. 

In [68]:
#Fix district column 
for i in range(0,len(reps)):
    reps['District'][i] = reps['District'][i].split("\xa0")[0]
reps.District.unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype=object)

In [69]:
#rename Born[2] column 
reps.rename(columns = {'Born[2]':'Birth_date'}, inplace = True)
reps.head()

,District,Member,Party,Prior experience,Education,Assumed office,Residence,Birth_date
0,Alabama,Jerry Carl,Republican,Mobile County Commission,Florida Gateway College,2021,Mobile,"June 17, 1958(age 64)"
1,Alabama,Barry Moore,Republican,Alabama House of Representatives,Enterprise State Community College (AS)\r\nAub...,2021,Enterprise,"September 26, 1966(age 56)"
2,Alabama,Mike Rogers,Republican,Calhoun County Commissioner\r\nAlabama House o...,"Jacksonville State University (BA, MPA)\r\nBir...",2003,Anniston,"July 16, 1958(age 64)"
3,Alabama,Robert Aderholt,Republican,Haleyville Municipal Judge,University of North Alabama\r\nBirmingham–Sout...,1997,Haleyville,"July 22, 1965(age 57)"
4,Alabama,Mo Brooks,Republican,Alabama House of Representatives\r\nMadison Co...,Duke University (BA)\r\nUniversity of Alabama ...,2011,Huntsville,"April 29, 1954(age 68)"


In [70]:
#Create Age column from Born[2] column 
for i in range(0,len(reps)):
    if reps['Birth_date'][i] != 'VACANT':
        tmp = reps['Birth_date'][i].split("(")
        tmp = str(tmp[1]).split("\xa0")[1][:-1]
        reps['Birth_date'][i] = tmp
    else:
        #drop row since house position currently VANCANT 
        reps = reps.drop(i)
reps.Birth_date.unique()

array(['64', '56', '57', '68', '49', '76', '72', '74', '63', '60', '42',
       '52', '65', '54', '62', '58', '77', '66', '71', '78', '36', '70',
       '82', '41', '46', '79', '45', '69', '59', '67', '43', '85', '53',
       '47', '50', '81', '61', '84', '48', '33', '38', '35', '40', '34',
       '44', '73', '55', '75', '39', '51', '83', '37', '32', '27', '86',
       '80'], dtype=object)

In [71]:
reps.head()

,District,Member,Party,Prior experience,Education,Assumed office,Residence,Birth_date
0,Alabama,Jerry Carl,Republican,Mobile County Commission,Florida Gateway College,2021,Mobile,64
1,Alabama,Barry Moore,Republican,Alabama House of Representatives,Enterprise State Community College (AS)\r\nAub...,2021,Enterprise,56
2,Alabama,Mike Rogers,Republican,Calhoun County Commissioner\r\nAlabama House o...,"Jacksonville State University (BA, MPA)\r\nBir...",2003,Anniston,64
3,Alabama,Robert Aderholt,Republican,Haleyville Municipal Judge,University of North Alabama\r\nBirmingham–Sout...,1997,Haleyville,57
4,Alabama,Mo Brooks,Republican,Alabama House of Representatives\r\nMadison Co...,Duke University (BA)\r\nUniversity of Alabama ...,2011,Huntsville,68


In [72]:
#Save complementary data set 
reps.to_csv(".\Cleaned_List_of_Representatives.csv")

Now having cleaned this dataset I would continue my analysis on Tableu Public.